In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import os

In [2]:
os.chdir(os.path.join(os.getcwd(),"03"))

In [3]:
# load datasets
data_train = pd.read_csv("sales_data_training.csv",dtype=np.float32)
data_test = pd.read_csv("sales_data_test.csv",dtype=np.float32)

In [4]:
data_train.head()

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0,132717.0,59.990002
1,4.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,83407.0,49.990002
2,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62423.0,49.990002
3,4.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,69889.0,39.990002
4,4.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,161382.0,59.990002


In [5]:
# train and test data split
X_train = data_train.drop("total_earnings",axis=1).values
y_train = data_train["total_earnings"].values

X_test = data_test.drop("total_earnings",axis=1).values
y_test = data_test["total_earnings"].values

In [6]:
# scaler mix max data 
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

In [7]:
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.fit_transform(X_test,y_test)

y_train = Y_scaler.fit_transform(y_train)
y_test  = Y_scaler.fit_transform(y_test)

C:\Users\Thor\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Thor\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Thor\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single featur

(400,)

In [35]:
y_train = np.array(y_train).reshape((1000,1))
y_test = np.array(y_test).reshape((400,1))

In [23]:
print(X_train.shape)
print(y_train.shape)

(1000, 9)
(1000, 1)


In [49]:
# Define model parameters
learning_rate = 0.001
training_epochs = 100

# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Section One: Define the layers of the neural network itself

# Input Layer
with tf.variable_scope('input',reuse=True):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

# Layer 1
with tf.variable_scope('layer_1',reuse=True):
    weights = tf.get_variable("weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2',reuse=True):
    weights = tf.get_variable("weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3',reuse=True):
    weights = tf.get_variable("weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output',reuse=True):
    weights = tf.get_variable("weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# Section Two: Define the cost function of the neural network that will measure prediction accuracy during training

with tf.variable_scope('cost',reuse=True):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train',reuse=True):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):
        

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_train, Y: y_train})

        # Print the current training status to the screen
        if epoch%5==0:
            training_cost = session.run(cost,feed_dict={X:X_train,Y:y_train})
            test_cost = session.run(cost,feed_dict={X:X_test,Y:y_test})
            print("epoch :",epoch, "training cost",training_cost,"testing cost",test_cost )

    # Training is now complete!
    print("Training is complete!")

epoch : 0 training cost 0.0517491 testing cost 0.0730912
epoch : 5 training cost 0.0252404 testing cost 0.0277302
epoch : 10 training cost 0.0107709 testing cost 0.0159939
epoch : 15 training cost 0.00845297 testing cost 0.0164164
epoch : 20 training cost 0.00439914 testing cost 0.00856019
epoch : 25 training cost 0.00344171 testing cost 0.00443791
epoch : 30 training cost 0.00202823 testing cost 0.00464839
epoch : 35 training cost 0.0013946 testing cost 0.00354422
epoch : 40 training cost 0.0010553 testing cost 0.00199594
epoch : 45 training cost 0.000778312 testing cost 0.00290895
epoch : 50 training cost 0.000531356 testing cost 0.00191234
epoch : 55 training cost 0.000381215 testing cost 0.0017685
epoch : 60 training cost 0.000316999 testing cost 0.00179113
epoch : 65 training cost 0.00026808 testing cost 0.00140467
epoch : 70 training cost 0.0002294 testing cost 0.00163038
epoch : 75 training cost 0.000202307 testing cost 0.00130429
epoch : 80 training cost 0.000179746 testing cos